In [1]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [4]:
MicrosoftR = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Microsoft/Microsoft_Ferdig.csv")

In [5]:
MicrosoftR.shape

(1812, 16)

In [6]:
MicrosoftR.head(3)

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,author_link,article_content,appears_in,symbols,related_articles,Time.1
0,2,2018-03-13,18:42:00,2,473,{'$oid': '5ab8d21e3efc2c03c520d00f'},https://www.nasdaq.com/article/is-oracle-orcl-...,Is Oracle (ORCL) Set to Beat Earnings Estimate...,{'$date': '2018-03-12T18:42:00Z'},"By Benjamin Rains, Zacks.com",https://www.nasdaq.com/author/zacks,Cloud computing has been one of the most well-...,"Investing,Earnings","AMZN,GOOGL,WDAY,MSFT,CRM",NaN,{'$date': '2018-03-12T18:42:00Z'}
1,4,2018-03-13,22:09:00,4,849,{'$oid': '5ab8d1dc3efc2c03c520cf8b'},https://www.nasdaq.com/article/1-smart-artific...,1 Smart Artificial Intelligence Bet You Can Ma...,{'$date': '2018-03-12T22:09:00Z'},"By Harsh Chauhan, Motley Fool",https://www.nasdaq.com/author/fool,"A report by IDC, which follows the tech indust...","PersonalFinance,Stocks","MSFT,AI,GOOGL,AMZN,GOOG",NaN,{'$date': '2018-03-12T22:09:00Z'}
2,6,2018-03-15,16:44:00,6,1313,{'$oid': '5ab8decb3efc2c03c520ec83'},https://www.nasdaq.com/article/3-large-cap-etf...,3 Large-Cap ETFs With Solid Inflows in March,{'$date': '2018-03-14T16:44:00Z'},"By Zacks Equity Research, Zacks.com",https://www.nasdaq.com/author/zacks,Wall Street has been recovering lately as fear...,"Investing,ETFs","AMZN,GOOGL,AAPL,MSFT,VV",NaN,{'$date': '2018-03-14T16:44:00Z'}


In [7]:
MicrosoftAksje = pd.read_csv(r"/Users/joakimlarsen/Documents/MasterOppgave/Microsoft/Microsoft_Prices_Label.csv")

In [8]:
MicrosoftAksje.shape

(8710, 8)

In [9]:
MicrosoftAksje.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume,Label
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.062055,1031788800,1
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.064271,308160000,1
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.065379,133171200,1


In [10]:
#Merge datasettene
merge = MicrosoftR.merge(MicrosoftAksje, how = "inner", on = "Date")

In [11]:
#Create a function to get the subjecticity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a fuction to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [12]:
#Create two new columns "Subjectivity" and "Polarity"
merge["Subjectivity"] = merge["article_content"].apply(getSubjectivity)
merge["Polarity"] = merge["article_content"].apply(getPolarity)

In [13]:
import nltk

from nltk.corpus import stopwords
stop = stopwords.words('english')

In [14]:
merge['article_content_without_stopwords'] = merge['article_content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [15]:
from nltk.stem import PorterStemmer

port = PorterStemmer()

In [16]:
def stem_words(article_content_without_stopwords):
    return " ".join([port.stem(word) for word in article_content_without_stopwords.split()])

merge["EtterStemming"] = merge["article_content_without_stopwords"].apply(lambda article_content_without_stopwords: stem_words(article_content_without_stopwords))
merge.head()

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,High,Low,Close,Adj Close,Volume,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming
0,2,2018-03-13,18:42:00,2,473,{'$oid': '5ab8d21e3efc2c03c520d00f'},https://www.nasdaq.com/article/is-oracle-orcl-...,Is Oracle (ORCL) Set to Beat Earnings Estimate...,{'$date': '2018-03-12T18:42:00Z'},"By Benjamin Rains, Zacks.com",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.475277,0.108063,Cloud computing one well-known drivers growth ...,cloud comput one well-known driver growth tech...
1,4,2018-03-13,22:09:00,4,849,{'$oid': '5ab8d1dc3efc2c03c520cf8b'},https://www.nasdaq.com/article/1-smart-artific...,1 Smart Artificial Intelligence Bet You Can Ma...,{'$date': '2018-03-12T22:09:00Z'},"By Harsh Chauhan, Motley Fool",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.440061,0.169724,"A report IDC, follows tech industry, estimates...","a report idc, follow tech industry, estim worl..."
2,10,2018-03-13,16:12:00,10,1660,{'$oid': '5ab8d2b83efc2c03c520d159'},https://www.nasdaq.com/article/3-strong-buy-se...,3 Strong Buy Semiconductor Stocks to Consider Now,{'$date': '2018-03-12T16:12:00Z'},"By Ryan McQueeney, Zacks.com",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.489353,0.146362,Semiconductor stocks battered recent market se...,semiconductor stock batter recent market sell-...
3,8,2018-03-13,07:05:00,8,1472,{'$oid': '5ab8d0693efc2c03c520cc37'},https://www.nasdaq.com/article/slack-needs-to-...,Slack Needs to Worry About Microsoft Teams,{'$date': '2018-03-13T07:05:00Z'},"By Timothy Green, Motley Fool",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.508590,0.175829,"Slack, software company focused workplace mess...","slack, softwar compani focus workplac messagin..."
4,6,2018-03-15,16:44:00,6,1313,{'$oid': '5ab8decb3efc2c03c520ec83'},https://www.nasdaq.com/article/3-large-cap-etf...,3 Large-Cap ETFs With Solid Inflows in March,{'$date': '2018-03-14T16:44:00Z'},"By Zacks Equity Research, Zacks.com",...,94.580002,92.830002,94.180000,90.933655,27611000,1,0.532905,0.115972,Wall Street recovering lately fears aggressive...,wall street recov late fear aggress rate hike ...


In [17]:
#Create a function to get the sentiment scores
def getSIA(EtterStemming):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(EtterStemming)
    return sentiment

In [18]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/joakimlarsen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [19]:
#Get the sentiment scores for each day
compound = []
neg = []
pos = []
neu = []
SIA = 0

for i in range(0, len(merge["EtterStemming"])):
    SIA = getSIA(merge["EtterStemming"][i])
    compound.append(SIA["compound"])
    neg.append(SIA["neg"])
    neu.append(SIA["neu"])
    pos.append(SIA["pos"])

In [20]:
#Store the sentiment scores in the merge data set:
merge["Compound"] = compound
merge["Negative"] = neg
merge["Neutral"] = neu
merge["Positive"] = pos

In [21]:
#Show the merge data
merge.head(3)

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,Volume,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming,Compound,Negative,Neutral,Positive
0,2,2018-03-13,18:42:00,2,473,{'$oid': '5ab8d21e3efc2c03c520d00f'},https://www.nasdaq.com/article/is-oracle-orcl-...,Is Oracle (ORCL) Set to Beat Earnings Estimate...,{'$date': '2018-03-12T18:42:00Z'},"By Benjamin Rains, Zacks.com",...,35387800,0,0.475277,0.108063,Cloud computing one well-known drivers growth ...,cloud comput one well-known driver growth tech...,0.9633,0.000,0.901,0.099
1,4,2018-03-13,22:09:00,4,849,{'$oid': '5ab8d1dc3efc2c03c520cf8b'},https://www.nasdaq.com/article/1-smart-artific...,1 Smart Artificial Intelligence Bet You Can Ma...,{'$date': '2018-03-12T22:09:00Z'},"By Harsh Chauhan, Motley Fool",...,35387800,0,0.440061,0.169724,"A report IDC, follows tech industry, estimates...","a report idc, follow tech industry, estim worl...",0.9892,0.048,0.823,0.130
2,10,2018-03-13,16:12:00,10,1660,{'$oid': '5ab8d2b83efc2c03c520d159'},https://www.nasdaq.com/article/3-strong-buy-se...,3 Strong Buy Semiconductor Stocks to Consider Now,{'$date': '2018-03-12T16:12:00Z'},"By Ryan McQueeney, Zacks.com",...,35387800,0,0.489353,0.146362,Semiconductor stocks battered recent market se...,semiconductor stock batter recent market sell-...,0.9948,0.016,0.844,0.140


In [22]:
#Create a list of column to keep
keep_columns = ["Open", "Low", "High", "Volume", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df = merge[keep_columns]
df

,Open,Low,High,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,97.000000,93.970001,97.239998,35387800,0.475277,0.108063,0.9633,0.000,0.901,0.099,0
1,97.000000,93.970001,97.239998,35387800,0.440061,0.169724,0.9892,0.048,0.823,0.130,0
2,97.000000,93.970001,97.239998,35387800,0.489353,0.146362,0.9948,0.016,0.844,0.140,0
3,97.000000,93.970001,97.239998,35387800,0.508590,0.175829,0.9906,0.062,0.803,0.135,0
4,93.529999,92.830002,94.580002,27611000,0.532905,0.115972,0.9877,0.074,0.781,0.145,1
...,...,...,...,...,...,...,...,...,...,...,...
1423,31.620001,31.250000,31.650000,63213000,0.461096,0.078854,0.9698,0.041,0.830,0.129,0
1424,32.869999,32.150002,32.869999,40685000,0.373844,-0.029014,0.7269,0.029,0.924,0.047,0
1425,37.240002,37.060001,38.220001,88948800,0.343695,0.045125,0.9300,0.040,0.867,0.094,1
1426,62.500000,62.240002,63.419998,35718900,0.400040,0.049240,0.7184,0.061,0.840,0.099,1


In [43]:
df.to_csv("MicrosoftFørTrening")

In [23]:
#Create the feature data set
X = df
X = np.array(X.drop(["Label"], 1))

#Create the target data set
y = np.array(df["Label"])

/var/folders/m7/01kjn8yx6d3_tjkcr1dkd9g80000gn/T/ipykernel_36723/988765945.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(X.drop(["Label"], 1))


In [41]:
X

array([[9.7000000e+01, 9.3970001e+01, 9.7239998e+01, ..., 9.0100000e-01,
        9.9000000e-02, 4.7300000e+02],
       [9.7000000e+01, 9.3970001e+01, 9.7239998e+01, ..., 8.2300000e-01,
        1.3000000e-01, 8.4900000e+02],
       [9.7000000e+01, 9.3970001e+01, 9.7239998e+01, ..., 8.4400000e-01,
        1.4000000e-01, 1.6600000e+03],
       ...,
       [3.7240002e+01, 3.7060001e+01, 3.8220001e+01, ..., 8.6700000e-01,
        9.4000000e-02, 3.5704100e+05],
       [6.2500000e+01, 6.2240002e+01, 6.3419998e+01, ..., 8.4000000e-01,
        9.9000000e-02, 3.5785700e+05],
       [6.0860001e+01, 6.0220001e+01, 6.1180000e+01, ..., 8.1400000e-01,
        1.6900000e-01, 3.8402600e+05]])

In [42]:
y

array([0, 0, 0, ..., 1, 1, 0])

In [24]:
#Lager for hele settet
Drop = ["Open", "Low", "High", "Volume", "Subjectivity", "Polarity", "Compound", "Negative", "Close", "Neutral", "Positive", "X", "Label"]
dx = merge[Drop]
#Create the feature data set
X = dx
X = np.array(X.drop(["Label"], 1))

#Create the target data set
y = np.array(dx["Label"])

/var/folders/m7/01kjn8yx6d3_tjkcr1dkd9g80000gn/T/ipykernel_36723/1931845828.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(X.drop(["Label"], 1))


In [25]:
#Split the data into 80% trainig and 20% testing data set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [26]:
len(x_test)

286

In [27]:
len(x_train)

1142

In [28]:
#Create and train the model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [29]:
#Show the models predictions
predictions = model.predict(x_test)
predictions

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0])

In [30]:
y_test

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0])

In [31]:
#Show the model metrics
print( classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94       140
           1       0.98      0.91      0.94       146

    accuracy                           0.94       286
   macro avg       0.95      0.94      0.94       286
weighted avg       0.95      0.94      0.94       286



In [32]:
#Create and train the model (Naive Bayes)
from sklearn.naive_bayes import GaussianNB
modelNB = GaussianNB()

In [33]:
#Train the model
modelNB.fit(x_train, y_train)

GaussianNB()

In [34]:
#Accuracy for modellen
modelNB.score(x_test, y_test)

0.5594405594405595

In [35]:
x_test[:10]

array([[ 5.78899990e+01,  5.68899990e+01,  5.80200000e+01,
         2.64974000e+07,  4.79085498e-01,  1.28204033e-01,
         9.91900000e-01,  4.70000000e-02,  5.71899990e+01,
         8.19000000e-01,  1.34000000e-01,  7.29460000e+04],
       [ 4.90200000e+01,  4.86699980e+01,  5.02400020e+01,
         4.67405000e+07,  5.16717931e-01,  1.35121896e-01,
         9.83900000e-01,  1.20000000e-02,  4.92799990e+01,
         7.70000000e-01,  2.18000000e-01,  1.77002000e+05],
       [ 5.04100000e+01,  4.98200000e+01,  5.04300000e+01,
         4.57105000e+07,  4.17602556e-01,  3.55148423e-02,
        -5.70700000e-01,  9.70000000e-02,  5.01300010e+01,
         8.09000000e-01,  9.40000000e-02,  2.28128000e+05],
       [ 5.49199980e+01,  5.34500010e+01,  5.49700010e+01,
         6.76255000e+07,  5.58333333e-01,  6.11111111e-02,
         2.96000000e-01,  0.00000000e+00,  5.34900020e+01,
         9.37000000e-01,  6.20000000e-02,  1.75689000e+05],
       [ 5.22999990e+01,  5.22500000e+01,  5.4070000

In [36]:
#Faktisk oppgang eller nedgang
y_test[:100]

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1])

In [37]:
#Sjekker om vi modellen predikerer rett
modelNB.predict(x_test[:100])

array([1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1])

In [38]:
#Sjekker sannsynligheten for oppgang og nedgang
Bayes = modelNB.predict_proba(x_test)

In [39]:
dx.head(22)

,Open,Low,High,Volume,Subjectivity,Polarity,Compound,Negative,Close,Neutral,Positive,X,Label
0,97.000000,93.970001,97.239998,35387800,0.475277,0.108063,0.9633,0.000,94.410004,0.901,0.099,473,0
1,97.000000,93.970001,97.239998,35387800,0.440061,0.169724,0.9892,0.048,94.410004,0.823,0.130,849,0
2,97.000000,93.970001,97.239998,35387800,0.489353,0.146362,0.9948,0.016,94.410004,0.844,0.140,1660,0
3,97.000000,93.970001,97.239998,35387800,0.508590,0.175829,0.9906,0.062,94.410004,0.803,0.135,1472,0
4,93.529999,92.830002,94.580002,27611000,0.532905,0.115972,0.9877,0.074,94.180000,0.781,0.145,1313,1
5,93.529999,92.830002,94.580002,27611000,0.420084,0.053018,0.9882,0.077,94.180000,0.801,0.122,1390,1
6,93.529999,92.830002,94.580002,27611000,0.434691,0.191460,0.9941,0.052,94.180000,0.837,0.112,1752,1
7,93.529999,92.830002,94.580002,27611000,0.523148,-0.048148,0.2960,0.000,94.180000,0.937,0.062,2863,1
8,93.529999,92.830002,94.580002,27611000,0.472999,0.032323,0.9210,0.054,94.180000,0.850,0.097,2864,1
9,95.120003,93.500000,95.410004,32132000,0.391251,0.149804,0.7783,0.074,93.849998,0.839,0.087,2015,0


In [92]:
dx["Sannsynlighet"] = Bayes

ValueError: Length of values (286) does not match length of index (1428)

In [40]:
dx.sort_values("Negative", ascending = True)

,Open,Low,High,Volume,Subjectivity,Polarity,Compound,Negative,Close,Neutral,Positive,X,Label
0,97.000000,93.970001,97.239998,35387800,0.475277,0.108063,0.9633,0.000,94.410004,0.901,0.099,473,0
608,45.450001,44.250000,45.709999,82180300,0.500000,0.000000,0.0000,0.000,44.529999,1.000,0.000,152604,0
612,46.790001,45.990002,46.830002,63837000,0.558333,0.061111,0.2960,0.000,46.099998,0.937,0.062,189536,0
613,46.790001,45.990002,46.830002,63837000,0.500000,0.271429,0.8750,0.000,46.099998,0.800,0.200,234904,0
615,39.000000,39.000000,39.790001,34330200,0.737446,0.120563,0.7003,0.000,39.209999,0.876,0.124,103620,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,46.500000,45.740002,46.799999,34422800,0.464222,0.160241,-0.9541,0.141,45.849998,0.730,0.130,191775,0
144,62.619999,62.349998,62.869999,19422300,0.297033,-0.001957,-0.9709,0.143,62.700001,0.822,0.034,22719,1
1321,37.650002,37.509998,38.139999,27195600,1.000000,0.000000,-0.7430,0.153,37.700001,0.847,0.000,365051,1
776,43.950001,43.799999,44.090000,29759800,0.489920,0.157096,-0.9782,0.153,43.990002,0.727,0.120,138961,1
